In [ ]:
import matplotlib

%matplotlib inline

In [ ]:
import glob
import boost_histogram as bh
import networkx as nx

In [ ]:
import uproot
import mplhep

mplhep.style.use("CMS")
import awkward
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sample = "SinglePiMinusFlatPt0p7To1000_cfi"

In [ ]:
!ls /hdfs/local/joosep/mlpf/gen/v2/

In [ ]:
tt = uproot.lazy(glob.glob("/hdfs/local/joosep/mlpf/gen/v2/{}/root/pfntuple_*.root".format(sample)))

In [ ]:
g = nx.DiGraph()
iev = 0

for i in range(len(tt.gen_pdgid[iev])):
    if tt.gen_status[iev][i] == 1:
        g.add_node(
            ("gen", i),
            typ=tt.gen_pdgid[iev][i],
            e=tt.gen_energy[iev][i],
            eta=tt.gen_eta[iev][i],
            phi=tt.gen_phi[iev][i],
        )

for i in range(len(tt.trackingparticle_pid[iev])):
    g.add_node(
        ("tp", i),
        typ=tt.trackingparticle_pid[iev][i],
        e=tt.trackingparticle_energy[iev][i],
        eta=tt.trackingparticle_eta[iev][i],
        phi=tt.trackingparticle_phi[iev][i],
    )

for i in range(len(tt.caloparticle_pid[iev])):
    g.add_node(
        ("cp", i),
        typ=tt.caloparticle_pid[iev][i],
        e=tt.caloparticle_energy[iev][i],
        eta=tt.caloparticle_eta[iev][i],
        phi=tt.caloparticle_phi[iev][i],
    )

for i in range(len(tt.element_type[iev])):
    g.add_node(
        ("el", i),
        typ=tt.element_type[iev][i],
        e=tt.element_energy[iev][i],
        eta=tt.element_eta[iev][i],
        phi=tt.element_phi[iev][i],
    )

for i in range(len(tt.pfcandidate_pdgid[iev])):
    g.add_node(
        ("pf", i),
        typ=tt.pfcandidate_pdgid[iev][i],
        e=tt.pfcandidate_energy[iev][i],
        eta=tt.pfcandidate_eta[iev][i],
        phi=tt.pfcandidate_phi[iev][i],
    )

for itp, ielem in zip(
    list(tt["trackingparticle_to_element.first"][iev]), list(tt["trackingparticle_to_element.second"][iev])
):
    g.add_edge(("tp", itp), ("el", ielem))

for itp, ielem in zip(list(tt["caloparticle_to_element.first"][iev]), list(tt["caloparticle_to_element.second"][iev])):
    g.add_edge(("cp", itp), ("el", ielem))

for ielem, ipf in zip(list(tt["element_to_candidate.first"][iev]), list(tt["element_to_candidate.second"][iev])):
    g.add_edge(("el", ielem), ("pf", ipf))

for icp in range(len(tt.caloparticle_idx_trackingparticle[iev])):
    itp = tt.caloparticle_idx_trackingparticle[iev][icp]
    if itp != -1:
        g.add_edge(("cp", icp), ("tp", itp))

In [ ]:
def node_label_func(n):
    return "{0} {1}\nE={2:.2f}\n{3:.1f}:{4:.1f}".format(
        n[0].upper(), g.nodes[n]["typ"], g.nodes[n]["e"], g.nodes[n]["eta"], g.nodes[n]["phi"]
    )


def node_color_func(n):
    colors = {"gen": "blue", "el": "gray", "pf": "purple", "tp": "red", "cp": "red", "gen": "blue"}
    return colors[n[0]]


node_labels = {n: node_label_func(n) for n in g.nodes}
node_colors = [node_color_func(n) for n in g.nodes]

In [ ]:
plt.figure(figsize=(50, 5))
pos = nx.nx_pydot.pydot_layout(g, prog="dot")
# pos = nx.spring_layout(g, k=0.15, iterations=20)
nx.draw_networkx(g, pos=pos, with_labels=False, node_color=node_colors, alpha=0.5)
nx.draw_networkx_labels(g, pos, labels=node_labels, font_size=4)
plt.savefig("{}_{}.svg".format(sample, iev))

In [ ]:
def plot_energy_stack(energies, pids):
    uniq_pids = np.unique(pids)
    hists = []
    bins = np.logspace(-1, 6, 61)
    for pid in uniq_pids:
        h = bh.Histogram(bh.axis.Variable(bins))
        h.fill(energies[pids == pid])
        hists.append(h)
    mplhep.histplot(hists, stack=False, label=[str(p) for p in uniq_pids])
    plt.legend()

In [ ]:
plot_energy_stack(awkward.flatten(tt.pfcandidate_energy), np.abs(awkward.flatten(tt.pfcandidate_pdgid)))
plt.xscale("log")
plt.yscale("log")
plt.title(sample + " PF")
plt.xlabel("$E_{PF}\ [GeV]$")
plt.savefig("{}_pf_energy.pdf".format(sample))

In [ ]:
plot_energy_stack(
    awkward.flatten(tt.gen_energy[tt.gen_status == 1]), np.abs(awkward.flatten(tt.gen_pdgid[tt.gen_status == 1]))
)
plt.xscale("log")
plt.yscale("log")
plt.title(sample + " Gen")
plt.xlabel("$E_{gen}\ [GeV]$")
plt.savefig("{}_gen_energy.pdf".format(sample))

In [ ]:
plot_energy_stack(awkward.flatten(tt.caloparticle_energy), np.abs(awkward.flatten(tt.caloparticle_pid)))
plt.xscale("log")
plt.yscale("log")
plt.title(sample + " CaloParticle")
plt.xlabel("$E_{CP}\ [GeV]$")
plt.savefig("{}_cp_energy.pdf".format(sample))

In [ ]:
plot_energy_stack(awkward.flatten(tt.trackingparticle_energy), np.abs(awkward.flatten(tt.trackingparticle_pid)))
plt.xscale("log")
plt.yscale("log")
plt.title(sample + " TrackingParticle")
plt.xlabel("$E_{TP}\ [GeV]$")
plt.savefig("{}_tp_energy.pdf".format(sample))

In [ ]:
bins = np.logspace(-3, 6, 101)
plt.hist(awkward.flatten(tt.pfcandidate_energy), bins=bins, histtype="step", lw=2, label="PF")
plt.hist(
    awkward.flatten(tt.gen_energy[tt.gen_status == 1]), bins=bins, histtype="step", lw=2, label="GenParticle\n(status=1)"
)
plt.hist(awkward.flatten(tt.caloparticle_energy), bins=bins, histtype="step", lw=2, label="CaloParticle")
# plt.hist(awkward.flatten(tt.trackingparticle_energy), bins=bins, histtype="step", lw=2, label="TrackingParticle")
plt.xscale("log")
plt.legend(loc="best")
plt.xlabel("E [GeV]")
plt.ylabel("Number of particles / bin")
plt.yscale("log")
plt.title(sample)
plt.savefig("{}_energy.pdf".format(sample))

In [ ]:
bins = np.logspace(-3, 6, 101)

pf_e = awkward.sum(tt.pfcandidate_energy, axis=-1)
gen_e = awkward.sum(tt.gen_energy[tt.gen_status == 1], axis=-1)
cp_e = awkward.sum(tt.caloparticle_energy, axis=-1)
# tp_e = awkward.sum(tt.trackingparticle_energy, axis=-1)

plt.hist(pf_e, bins=bins, histtype="step", lw=2, label="PF")
plt.hist(gen_e, bins=bins, histtype="step", lw=2, label="GenParticle\n(status=1)")
plt.hist(cp_e, bins=bins, histtype="step", lw=2, label="CaloParticle")
# plt.hist(tp_e, bins=bins, histtype="step", lw=2, label="TrackingParticle")

plt.xscale("log")
plt.legend(loc=2)
plt.xlabel("$\sum E\ [GeV]$")
plt.ylabel("Number of events / bin")
# plt.yscale("log")
plt.title(sample)
plt.savefig("{}_sum_energy.pdf".format(sample))

In [ ]:
plt.scatter(gen_e, pf_e, alpha=0.7, label="x=PF")
plt.scatter(gen_e, cp_e, alpha=0.7, label="x=CaloParticle")
# plt.scatter(gen_e, tp_e, alpha=0.7)
plt.xlabel("$\sum E_{gen}\ [GeV]$")
plt.ylabel("$\sum E_x\ [GeV]$")
plt.xscale("log")
plt.yscale("log")
plt.legend(loc="best")
plt.title(sample)
plt.savefig("{}_energy_corr.pdf".format(sample))

In [ ]:
bins = np.linspace(0, 4, 100)
plt.hist(gen_e / pf_e, bins=bins, histtype="step", lw=2, label="x=GenParticle\n(status=1)")
plt.hist(cp_e / pf_e, bins=bins, histtype="step", lw=2, label="x=CaloParticle")
# plt.hist(tp_e/pf_e, bins=bins, histtype="step", lw=2, label="TrackingParticle")
plt.legend(loc="best")

plt.legend(loc="best")
plt.xlabel("$\sum E_x / \sum E_{PF}$")
plt.title(sample)
plt.savefig("{}_energy_ratio.pdf".format(sample))